# Recap


Before we start, I will copy some idea that we got in previous stages :
#### Part 1

- Here we have 1383 building, 758 in training data and the rest in test data.
- We have 276923 training data, and 81827 test data
- We have many missing value in temperature
- etc

- In the data train each building is provided with the same number of sample trainings, namely samples in units of hours in 28 days. while in the data test, the number of data samples for each building varies, there are 4 days, 12 days, etc.

For more detail you can check direcly to the notebook https://github.com/fauzantaufik/Cold-Start-Energy-Forecasting/blob/master/EXPLOLATORY%20DATA%20TO%20DEFINE%20PROBLEM%20DEEPLY.ipynb

#### Part 2

- seen that consumption from year to year is increasing, so maybe years will be a good feature to be the input in our model
- The month will also be a good feature. there is a seasonal pattern from month to month.
- There is some noise in data
- Consumption nearly uniform from hour to hour, but some date have pattern in consomption by hour to hour
- There is some correlation bettwen temperature/hour and consumption/hour

"""One of the challenges is in the way we fill in the missing values. because many buildings have a temperature value that is lost up to 100%. meaning that for one year the temperature inside the building is unknown!!.

we cannot fill this missing value with linear interpolation, even if it has to be non-linear we will have difficulty determining the degree of non-linearity.

Because all series IDs have the same pattern regarding the temperature value. then it's have to be good if we try to build a model that will predict the temperature value. this is a gonna be like other project though, but it's worth it since we have missinig value almost 50% in temperature""""

=======================================

"""For electricity consumption in each building, it seems that each building has a different level of electricity consumption. may depend on the type of building. there are buildings that always need electricity every time. there are also buildings that only require electricity when at certain times such as working hours and then a little consumption at night."""

Talk about electricity consumption : 
- There is a changes in consumption pattern over year. but it's not abvious the rule of pattern. and I dont know is it significant or not. but it's in my hipotesis.
- There is no really abvious pattern of electricity consumption over month and day of month.
- There is a pattern in the name of day, the consumpton is low in holiday (saturday and sunday) than in work days.
- There is a pattern in series of hours, which is the high consumption at work time.

We need to talk about Temperature due to we wanna build predictive model for temperature because of the need of fill in the missing value :
- The temperature seem to be increase over year.
- The temperature seem to be have seasonal pattern in series of month.
- The temperature seem to be constant in the day of month and in the day of week.
- Of course, the temperature is higher in the daytime hours

- the variance in consumption vs. lag increases with further lag
- temperature vs temperature lag-k seem have to be same 

One more problem is we have no sample for electricity consumption and temperature in 2012.

For more detail you check the notebook : https://github.com/fauzantaufik/Cold-Start-Energy-Forecasting/blob/master/Visualization%20Cold%20Start%20Energy%20Forecasting.ipynb

# Feature Engeenering

Substep of Feature Engeenering :

1. Feature Creation

    1.1 Feature Extractin of Time (Years, Months, Day of Week, Hours)
    
    1.2 Feature Over Time
    
    1.3 Adding Lag Feature
    
    1.4 Adding SMA (smooth moving avarage), pct_change
    
    1.5 Adding National Holiday Feature
    
    1.6 Adding meta data information
    
    
2. Data Cleaning

    2.1 Build Predictive Model for temperature
    
    2.2 Filling missing value of temperature by Predictive Model that have been build
    
    2.3 Cleaning Outlier
    
    
3. Feature Selection
    
    3.1 Inspecting by Heatmap

    3.2 Find coeffecient with linear model first
    
    3.3 RFECV
    
    


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
meta_data = pd.read_csv('meta.csv')
train_data = pd.read_csv('consumption_train.csv')
test_data = pd.read_csv('cold_start_test.csv')
submission = pd.read_csv('submission_format.csv')

In [3]:
submission[submission.prediction_window=='hourly'].head()

,pred_id,series_id,timestamp,temperature,consumption,prediction_window
31,31,101878,2014-04-08 00:00:00,18.85,0.0,hourly
32,32,101878,2014-04-08 01:00:00,19.00,0.0,hourly
33,33,101878,2014-04-08 02:00:00,19.00,0.0,hourly
34,34,101878,2014-04-08 03:00:00,18.15,0.0,hourly
35,35,101878,2014-04-08 04:00:00,17.00,0.0,hourly


In [4]:
test_data[test_data.consumption==0]

,Unnamed: 0,series_id,timestamp,consumption,temperature
4261,4261,101464,2013-11-05 13:00:00,0.0,22.15
4262,4262,101464,2013-11-05 14:00:00,0.0,23.00
78730,78730,100721,2012-06-05 10:00:00,0.0,NaN
78731,78731,100721,2012-06-05 11:00:00,0.0,NaN


In [5]:
test_data[test_data.series_id==100721].head()

,Unnamed: 0,series_id,timestamp,consumption,temperature
78624,78624,100721,2012-06-01 00:00:00,80921.671453,NaN
78625,78625,100721,2012-06-01 01:00:00,63264.995507,NaN
78626,78626,100721,2012-06-01 02:00:00,61655.566815,NaN
78627,78627,100721,2012-06-01 03:00:00,69971.508388,NaN
78628,78628,100721,2012-06-01 04:00:00,100214.655903,NaN


In [5]:
submission.prediction_window.unique()

array(['daily', 'weekly', 'hourly'], dtype=object)

In [6]:
set(submission.series_id) == set(test_data.series_id)

True

In [7]:
"""Change the types of timestamp columns and set the index"""

train_data.timestamp = pd.to_datetime(train_data.timestamp)
test_data.timestamp = pd.to_datetime(test_data.timestamp)

train_data.index = train_data.timestamp
test_data.index = test_data.timestamp

train_data.drop('timestamp', axis=1, inplace=True)
train_data.drop('Unnamed: 0', axis=1, inplace=True)

test_data.drop('timestamp', axis=1, inplace=True)
test_data.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
meta_data.head()

,series_id,surface,base_temperature,monday_is_day_off,tuesday_is_day_off,wednesday_is_day_off,thursday_is_day_off,friday_is_day_off,saturday_is_day_off,sunday_is_day_off
0,100003,x-large,low,False,False,False,False,False,True,True
1,100004,x-large,low,False,False,False,False,False,True,True
2,100006,x-small,low,False,False,False,False,False,True,True
3,100008,x-small,low,False,False,False,False,False,True,True
4,100010,x-small,low,False,False,False,False,False,True,True


In [9]:
day_off = meta_data.columns[3:]

count_day_off = {'{} count'.format(day) : meta_data[day].sum() for day in day_off}

count_day_off

{'friday_is_day_off count': 17,
 'monday_is_day_off count': 4,
 'saturday_is_day_off count': 1258,
 'sunday_is_day_off count': 1274,
 'thursday_is_day_off count': 7,
 'tuesday_is_day_off count': 4,
 'wednesday_is_day_off count': 7}

In [10]:
import statsmodels.api as sm

def evaluate_feature_rf(x, y, metrics) :
    model_evaluate = RandomForestRegressor(criterion='mae')
    model_evaluate.fit(x, y)
    score = metrics(y, model_evaluate.predict(x))
    
    plt.barh(model_evaluate.feature_importances_, x.columns)
    plt.title('Feature Importances')
    plt.show()
    
    print('Score : {}'.format(score))
    
def evaluate_feature_dtr(x, y, metrics) :
    model_evaluate = DecisionTreeRegressor(criterion='mae')
    model_evaluate.fit(x, y)
    score = metrics(y, model_evaluate.predict(x))
    
    plt.barh(model_evaluate.feature_importances_, x.columns)
    plt.title('Feature Importances')
    plt.show()
    
    print('Score : {}'.format(score))
    
def feature_summary(x, y, metrics) :
    linear_features = sm.add_constant(x)
    model = sm.OLS(y, x)
    results = model.fit()  # fit the model
    
    print(results.summary())
    print(results.pvalues)
    
    prediction = results.predict(x)
    score = metrics(y.values, prediction)
    print("Mean Absolute Error {}".format(score))

### 1.1 Feature Extraction of Time

In [11]:
train_data['year'] = train_data.index.year
train_data['month'] = train_data.index.month_name()
train_data['dayofweek'] = train_data.index.day_name()
train_data['hour'] = train_data.index.hour

test_data['year'] = test_data.index.year
test_data['month'] = test_data.index.month_name()
test_data['dayofweek'] = test_data.index.day_name()
test_data['hour'] = test_data.index.hour

In [12]:
train_data.head()

,series_id,consumption,temperature,year,month,dayofweek,hour
timestamp,,,,,,,
2014-12-24 00:00:00,103088,101842.233424,NaN,2014,December,Wednesday,0
2014-12-24 01:00:00,103088,105878.048906,NaN,2014,December,Wednesday,1
2014-12-24 02:00:00,103088,91619.105008,NaN,2014,December,Wednesday,2
2014-12-24 03:00:00,103088,94473.706203,NaN,2014,December,Wednesday,3
2014-12-24 04:00:00,103088,96976.755526,NaN,2014,December,Wednesday,4


In [13]:
test_data.head()

,series_id,consumption,temperature,year,month,dayofweek,hour
timestamp,,,,,,,
2013-02-27 00:00:00,102781,15295.740389,17.00,2013,February,Wednesday,0
2013-02-27 01:00:00,102781,15163.209562,18.25,2013,February,Wednesday,1
2013-02-27 02:00:00,102781,15022.264079,18.00,2013,February,Wednesday,2
2013-02-27 03:00:00,102781,15370.420458,17.00,2013,February,Wednesday,3
2013-02-27 04:00:00,102781,15303.103213,16.90,2013,February,Wednesday,4


In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 509376 entries, 2014-12-24 00:00:00 to 2017-12-18 23:00:00
Data columns (total 7 columns):
series_id      509376 non-null int64
consumption    509376 non-null float64
temperature    280687 non-null float64
year           509376 non-null int64
month          509376 non-null object
dayofweek      509376 non-null object
hour           509376 non-null int64
dtypes: float64(2), int64(3), object(2)
memory usage: 31.1+ MB


In [14]:
## Evaluation Feature Performance

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

train_data.year = train_data.year.astype('object')
train_data.hour = train_data.hour.astype('object')

feature_test = ['year', 'month', 'dayofweek', 'hour']
df_test=train_data[feature_test]
y= train_data.consumption

X = pd.get_dummies(df_test, drop_first=True)

C:\Users\Data Scientist\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [15]:
feature_summary(X, y, mean_absolute_error)

                            OLS Regression Results                            
Dep. Variable:            consumption   R-squared:                       0.334
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                     5793.
Date:                Sat, 27 Oct 2018   Prob (F-statistic):               0.00
Time:                        15:02:19   Log-Likelihood:            -6.8241e+06
No. Observations:              509376   AIC:                         1.365e+07
Df Residuals:                  509332   BIC:                         1.365e+07
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
year_2014            4.172e+04   1

### 1.2 Feature Over Time

In [34]:
temperature_rolling = train_data.dropna()['temperature'].rolling(24).agg(['mean', 'min', 'max', 'std'])
temperature_rolling = pd.concat([train_data.dropna().temperature, temperature_rolling], axis=1)
y = train_data.dropna()['consumption']

In [35]:
temperature_rolling.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 280687 entries, 2014-12-30 17:00:00 to 2017-12-14 23:00:00
Data columns (total 5 columns):
temperature    280687 non-null float64
mean           280664 non-null float64
min            280664 non-null float64
max            280664 non-null float64
std            280664 non-null float64
dtypes: float64(5)
memory usage: 12.8 MB


In [37]:
feature_summary(temperature_rolling.interpolate()[23:], y[23:], mean_absolute_error)

                            OLS Regression Results                            
Dep. Variable:            consumption   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.270
Method:                 Least Squares   F-statistic:                 2.077e+04
Date:                Sat, 27 Oct 2018   Prob (F-statistic):               0.00
Time:                        15:20:56   Log-Likelihood:            -3.7669e+06
No. Observations:              280664   AIC:                         7.534e+06
Df Residuals:                  280659   BIC:                         7.534e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
temperature  3134.3997     91.158     34.384      

In [44]:
train_data =train_data.join(temperature_rolling, rsuffix='_r')
train_data.drop('temperature_r', axis=1, inplace=True)

### 1.3 Adding Lag Feature

In [ ]:
shifts = np.arange(1, 24)

temperature_shifts = pd.DataFrame({'temperature_shift_{}'.format(n_shift) : train_data.temperature.shift(n_shift) for n_shift in
                                  shifts})

,series_id,consumption,temperature,year,month,dayofweek,hour,mean,min,max,std
timestamp,,,,,,,,,,,
2013-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02 00:00:00,103298.0,135802.353408,-3.766667,2013,January,Wednesday,0,11.975000,-3.766667,18.633333,4.562769
2013-01-02 00:00:00,103298.0,135802.353408,-3.766667,2013,January,Wednesday,0,11.355903,-3.766667,17.075000,4.798539
2013-01-02 00:00:00,102642.0,28843.506793,-3.766667,2013,January,Wednesday,0,11.975000,-3.766667,18.633333,4.562769
2013-01-02 01:00:00,102642.0,28843.506793,-3.766667,2013,January,Wednesday,0,11.355903,-3.766667,17.075000,4.798539
2013-01-02 01:00:00,103298.0,135586.622902,-3.833333,2013,January,Wednesday,1,11.290972,-3.833333,18.633333,5.583897
2013-01-02 01:00:00,103298.0,135586.622902,-3.833333,2013,January,Wednesday,1,10.876389,-3.833333,17.075000,5.676967
2013-01-02 01:00:00,102642.0,26445.243198,-3.833333,2013,January,Wednesday,1,11.290972,-3.833333,18.633333,5.583897
2013-01-02 02:00:00,102642.0,26445.243198,-3.833333,2013,January,Wednesday,1,10.876389,-3.833333,17.075000,5.676967
